In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Equity - Handling Corporate Actions

Attributes
----------
equity
split
merger
spin-off
bonus issue
dividend
recipes
valuations
corporate actions
"""

toggle_code("Toggle Docstring")

# Creation and Lifecycle of Corporate Actions

In this notebook, we demonstrate how you can create corporate actions and apply them to a portfolio.
We will also demonstrate how the holdings in the portfolio change as we go through the lifecycle of the corporate action.

## Table of Contents:
* [1. Upload Instrument Master](#1.-Upload-Instrument-Master)
* [2. Create a Corporate Action Source and Link to a Portfolio](#2.-Create-a-Corporate-Action-Source-and-Link-to-a-Portfolio)
* [3. Transactions](#3.-Transactions)
* [4. Creating a Corporate Action](#4.-Creating-a-Corporate-Action)
* [4.1 Dividend](#4.1-Dividend)
* [4.2 Merger](#4.2-Merger)
* [4.3 Spin-off](#4.3-Spin-off)
* [4.4 Split](#4.4-Split)
* [4.5 Bonus Issue](#4.5-Bonus-Issue)

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from corporate_actions_utilities import (
    run_portfolio_valuation,
    load_eod_prices,
    load_corporate_action_split_price,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_transactions_response,
)


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.0.1.0


We will first set up our APIs and variables as well as create 2 helper functions that we will use in this notebook.

In [3]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
corporate_action_sources_api = api_factory.build(lu.api.CorporateActionSourcesApi)
aggregation_api = api_factory.build(lu.AggregationApi)
quotes_api = api_factory.build(lu.QuotesApi)

In [4]:
# Define scopes
scope = "ibor"
ca_code = "ibor_corp_act"
portfolio_code = "CorporateActionsPortfolio"

In [5]:
def to_datetime(date):
    datetime_date = datetime.strptime(date, "%d/%m/%Y").replace(tzinfo=pytz.utc)
    return datetime_date


def get_portfolio_holdings(date):
    holdings = transaction_portfolios_api.get_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at=to_datetime(date),
        property_keys=["instrument/default/Name"],
    )

    return lusid_response_to_data_frame(holdings)[
        [
            "properties.Instrument/default/Name.value.label_value",
            "holding_type",
            "units",
            "settled_units",
            "cost.amount",
            "cost.currency",
            "cost_portfolio_ccy.amount",
        ]
    ]

## 1. Upload instrument master

Before creating a portfolio, we must first define the assets that we will be working with by uploading an instrument master.

In [6]:
instruments = pd.read_csv("data/corporateactions-instruments.csv")
instruments

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,NaN
3,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,NaN
4,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,NaN
5,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,NaN
6,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,NaN
7,ACM_Research_Inc,imd_23423410,USD,US0236530000,BBG00HPSG933,UN,united_states_america,ACMR,equity,common_stock,NaN


In [7]:
# Load the instrument into LUSID

instrument_identifier_mapping = {
    "Figi": "figi",
    "Isin": "isin",
}

instrument_mapping_required = {"name": "instrument_name"}
instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instruments,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
)

succ, failed, errors = format_instruments_response(responses)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,8,0,0


## 2. Create a Corporate Action Source and Link to a Portfolio

All corporate actions are stored in a corporate actions source. This source will then be linked to a portfolio using the corporate_action_source_id parameter upon creation of the portfolio. Alternatively, you can also update the corporate actions source of a portfolio after creating it by using https://www.lusid.com/docs/api/#operation/UpsertPortfolioDetails.

In [8]:
try:

    source_request = lm.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_code,
        display_name="Ibor Corporate Action Source",
        description="Corporate Actions source for sample notebook",
    )

    corporate_action_sources_api.create_corporate_action_source(
        create_corporate_action_source_request=source_request
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a Corporate Action Source with id ibor_corp_act because it already exists in scope ibor.


In [9]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
            corporate_action_source_id=lu.ResourceId(scope=scope, code=ca_code),
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'CorporateActionsPortfolio' because it already exists in scope 'ibor'.


## 3. Transactions

We populate our portfolio with transactions so we have assets to perform our corporate actions on.

In [10]:
# upsert transactions
transactions = pd.read_csv("data/corporate_actions_transactions.csv")
transactions

,portfolio_code,transaction_id,figi,ticker,name,transaction_type,transaction_date,settlement_date,transaction_units,transaction_price,total_consideration,transaction_currency
0,CorporateActionsPortfolio,TX001,BBG000B9XVV8,AAPL,Apple_Nasdaq_AAPL,StockIn,01/01/2018,02/01/2018,100,43.75,"4,375.00",USD
1,CorporateActionsPortfolio,TX002,BBG000C16621,ESRX,ExpressScripts_NYSE_ESRX,StockIn,01/01/2018,02/01/2018,100,76.01,"7,601.00",USD
2,CorporateActionsPortfolio,TX003,BBG000BVL406,TRN,TrinityIndustries_NYSE_TRN,StockIn,01/01/2018,02/01/2018,100,26.34,"2,634.00",USD
3,CorporateActionsPortfolio,TX004,BBG000BTGM43,TREX,Trex_NYSE_TREX,StockIn,01/01/2018,02/01/2018,100,27.83,"2,783.00",USD
4,CorporateActionsPortfolio,TX005,BBG00HPSG933,ACMR,ACM_Research_Inc,StockIn,01/01/2018,02/01/2018,100,2.00,200.00,USD


In [11]:
# map the transactions file

transaction_field_mapping_required = {
    "code": "portfolio_code",
    "transaction_id": "transaction_id",
    "type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "transaction_units",
    "transaction_price.price": "transaction_price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "total_consideration",
    "total_consideration.currency": "transaction_currency",
    "transaction_currency": "transaction_currency",
}


transaction_identifier_mapping = {
    "Figi": "figi",
}

In [12]:
responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transactions,
    mapping_required=transaction_field_mapping_required,
    mapping_optional={},
    identifier_mapping=transaction_identifier_mapping,
    file_type="transaction",
)

succ, failed = format_transactions_response(responses)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


## 4. Creating corporate actions

The below contains details of the corporate actions that we will be using for these examples. For details on how corporate actions are modelled in LUSID, see https://support.lusid.com/knowledgebase/article/KA-01867/en-us

In [13]:
ca_data = pd.read_csv("data/corporate_actions.csv")
ca_data

,Description,code,action_description,announcement_date,ex_date,record_date,payment_date,input_instrument_figi,instrument_input_ticker,input_instrument_name,input_units_factor,input_cost_factor,output_instrument_figi,output_instrument_name,output_ticker,output_instrument_internal,output_units_factor,output_cost_factor,optional_second_output_instrument_figi,optional_second_output_instrument_name,optional_second_output_ticker,optional_second_output_instrument_internal,optional_second_output_units_factor,optional_second_output_cost_factor
0,Apple Dividend of 0.17 cents per share,ca001,dividend,29/01/2019,08/02/2019,11/02/2019,14/02/2019,BBG000B9XVV8,AAPL,APPLE INC,1,1,NaN,NaN,NaN,CCY_USD,0.17,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1,Cigna & Express Scripts Merger,ca002,merger (equivalent to exchange example),08/03/2018,10/12/2018,13/12/2018,20/12/2018,BBG000C16621,ESRX,EXPRESS SCRIPTS HOLDINGS CO,1,1,BBG00KXXK940,CIGNA CORP,CI,NaN,0.24,1.00,NaN,NaN,NaN,NaN,NaN,NaN
2,Acosa spin-off from Trinity Industries,ca003,spin-off (equivalent to exchange example),01/01/2018,17/10/2018,20/10/2018,01/11/2018,BBG000BVL406,TRN,TRINITY INDUSTRIES INC,3,1,BBG00JGMWFQ5,ARCOSA INC,ACA,NaN,1.00,0.28,BBG000BVL406,TRN,TRINITY INDUSTRIES INC,NaN,1.00,0.72
3,Trex 2 for 1 stock split,ca004,split,07/05/2018,21/05/2018,23/05/2018,19/06/2018,BBG000BTGM43,TREX,TREX COMPANY INC,1,1,BBG000BTGM43,TREX COMPANY INC,TREX,NaN,2.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN
4,ACM Research Inc bonus issue,ca005,bonus issue,03/03/2022,14/03/2022,16/03/2022,23/03/2022,BBG00HPSG933,ACMR,ACM Research Inc,1,1,BBG00HPSG933,ACM Research Inc,ACMR,NaN,2.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN


### 4.1. Dividend

A dividend will give us a certain amount of money or shares for every share we currently hold in a company. In our scenario, we are getting 17 cents per share of Apple stock. We own 100 shares in Apple so we can expect to receive 17 USD. We can see that our input factor is 1 and our output factor is 0.17. This means that for every 1 unit of our input instrument (Apple shares) we will receive 0.17 units of our output instrument (USD).

In [14]:
dataset = ca_data.iloc[0]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset["input_instrument_figi"],
            },
            units_factor=float(dataset["input_units_factor"]),
            cost_factor=float(dataset["input_cost_factor"]),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={"Instrument/default/Currency": "USD"},
                units_factor=float(dataset["output_units_factor"]),
                cost_factor=float(dataset["output_cost_factor"]),
            )
        ],
    )
]

dividend_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset["code"],
    announcement_date=to_datetime(dataset["announcement_date"]),
    ex_date=to_datetime(dataset["ex_date"]),
    record_date=to_datetime(dataset["record_date"]),
    payment_date=to_datetime(dataset["payment_date"]),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[dividend_request]
)

On the announcement date of the dividend, we take a look at our holdings and see just a position in Apple of 100 units.

In [15]:
get_portfolio_holdings("29/01/2019")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00
5,Cigna_NYSE_CI,P,24.00,24.00,"7,601.00",USD,"7,601.00"


On the ex-date, we can see the USD cash dividend coming in but it has not settled yet. We can see this under settled_units which is still 0.

In [16]:
get_portfolio_holdings("08/02/2019")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00
5,Cigna_NYSE_CI,P,24.00,24.00,"7,601.00",USD,"7,601.00"
6,USD,A,17.00,0.00,17.00,USD,17.00


Finally on the payment date, we can see the USD cash position settled into our holdings.

In [17]:
get_portfolio_holdings("14/02/2019")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00
5,Cigna_NYSE_CI,P,24.00,24.00,"7,601.00",USD,"7,601.00"
6,USD,B,17.00,17.00,17.00,USD,17.00


### 4.2. Merger

In our example of the merger here, we will show you what happened to Express Scripts when it got acquired by Cigna. In this merger, the shareholders of Express Scripts received 0.24 shares in Cigna for every Express Scripts share they held. We can see that the output factor for this transaction is 0.24. Given that we owned 100 shares in Express Scripts at the time of this merger, we should receive 24 shares in Cigna.

In [18]:
dataset = ca_data.iloc[1]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset["input_instrument_figi"]
            },
            units_factor=float(dataset["input_units_factor"]),
            cost_factor=float(dataset["input_cost_factor"]),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset["output_instrument_figi"]
                },
                units_factor=float(dataset["output_units_factor"]),
                cost_factor=float(dataset["output_cost_factor"]),
            )
        ],
    )
]

merger_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset["code"],
    announcement_date=to_datetime(dataset["announcement_date"]),
    ex_date=to_datetime(dataset["ex_date"]),
    record_date=to_datetime(dataset["record_date"]),
    payment_date=to_datetime(dataset["payment_date"]),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[merger_request]
)

On the announcement date of the merger, we can see a position of 100 units in Express Scripts in our holdings.

In [19]:
get_portfolio_holdings("08/03/2018")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00


On the ex-date, we see an unsettled position of 24 units in Cigna as expected. These will then settle on 20/12/2018 which is the payment date.

In [20]:
get_portfolio_holdings("10/12/2018")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00
5,Cigna_NYSE_CI,P,24.00,0.00,"7,601.00",USD,"7,601.00"


### 4.3. Spin-off

In a spin-off the company will sell off a part of their business. In our example, the company will reward its shareholders with shares of the spun-off company. Owners of Trinity Industries shares will receive 1 share in Arcosa for every 3 shares they hold in Trinity Industries.

In [21]:
dataset = ca_data.iloc[2]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset["input_instrument_figi"]
            },
            units_factor=float(dataset["input_units_factor"]),
            cost_factor=float(dataset["input_cost_factor"]),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset["output_instrument_figi"]
                },
                units_factor=float(dataset["output_units_factor"]),
                cost_factor=float(dataset["output_cost_factor"]),
            ),
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset[
                        "optional_second_output_instrument_figi"
                    ]
                },
                units_factor=float(dataset["optional_second_output_units_factor"]),
                cost_factor=float(dataset["optional_second_output_cost_factor"]),
            ),
        ],
    )
]

spin_off_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset["code"],
    announcement_date=to_datetime(dataset["announcement_date"]),
    ex_date=to_datetime(dataset["ex_date"]),
    record_date=to_datetime(dataset["record_date"]),
    payment_date=to_datetime(dataset["payment_date"]),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[spin_off_request]
)

On the day of the announcement, we can see that we hold 100 shares in Trinity Industries.

In [22]:
get_portfolio_holdings("01/01/2018")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,0.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,0.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,0.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,100.00,0.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,0.00,200.00,USD,200.00


In this spin-off, we have an input units factor of 3 and an output units factor of 1. This means that for every 3 shares in Trinity Industries we hold, we will receive 1 share in Arcosa.
We can see this reflected in our holdings, as we had 100 units of Trinity Industries and we thus we receive 33 shares in Arcosa on the payment date.

In [23]:
get_portfolio_holdings("01/11/2018")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,34.00,34.00,"1,895.93",USD,"1,895.93"
3,Trex_NYSE_TREX,P,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,P,33.00,33.00,738.07,USD,738.07


### 4.4. Split

During a stock split, the company will multiply the amount of shares and divide the price by an equal amount. This causes them to lower the share price but keep the market cap of the company equal. In our example we have a 2 for 1 stock split, meaning shareholders get 2 shares for every share they own and their cost basis remains the same. In this scenario our input factor is 1 and our output factor is 2 while the cost factors remain 1 for both. As a result you get 2 times the output instrument for every 1 unit of input instrument.

In [24]:
dataset = ca_data.iloc[3]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset["input_instrument_figi"]
            },
            units_factor=float(dataset["input_units_factor"]),
            cost_factor=float(dataset["input_cost_factor"]),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset["output_instrument_figi"]
                },
                units_factor=float(dataset["output_units_factor"]),
                cost_factor=float(dataset["output_cost_factor"]),
            )
        ],
    )
]

split_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset["code"],
    announcement_date=to_datetime(dataset["announcement_date"]),
    ex_date=to_datetime(dataset["ex_date"]),
    record_date=to_datetime(dataset["record_date"]),
    payment_date=to_datetime(dataset["payment_date"]),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[split_request]
)

On the announcement date of this split (7th of May), we can see that we have 100 shares in Trex. The split will be 2 for 1 so we expect to have 200 shares on the Ex Date (21st of May).

In [25]:
get_portfolio_holdings("07/05/2018")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,100.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00


On the Ex Date, we can see that we have 200 units in Trex as we expected and our cost remains unaffected.

In [26]:
get_portfolio_holdings("21/05/2018")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,100.00,100.00,"2,634.00",USD,"2,634.00"
3,Trex_NYSE_TREX,P,200.00,100.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00


#### 4.4.1 Running valuations before and after splits

As part of the stock split, we also need to adjust the quotes in LUSID in proportion to the split. For example, after a 2 for 1 split, we will hold twice as many shares, but each is worth half the previous amount. Therefore we need a new price to account for this change. In the example below, we run two valuations using <b>Trex_NYSE_TREX</b> as an example:

* The first valuation occurs just before midnight on the 20th of May. At this point, the stock has not been split. We are using a price of USD 53.45 from the `Quotes` store.
* The stock is then spit in LUSID at midnight or 00:00:00 on 21st of May (this is the ExDate in LUSID)
* We then run another valuation at 8AM the next day (i.e. after the split), and we can see that the price has been adjusted to USD 26.73

The key message here is that while the units increase from 100 to 200 (due to the 2 for 1 split), the PV of the position remains unchanged. If we did not adjust the price, then the PV would be incorrect. 

To illustrate, we run a valuation just before the Trex stock is split in the IBOR. Take note of the PV and Units.

In [27]:
# Load some quotes to value the portfolio pre-split

load_eod_prices(scope)

In [28]:
run_portfolio_valuation(portfolio_code, "2018-05-20T23:59:59Z", scope)

,Instrument/default/Name,Proportion(Valuation/PvInReportCcy),Quotes/Price,Sum(Valuation/PvInReportCcy),Sum(Holding/default/Units)
0,Apple_Nasdaq_AAPL,0.36,131.33,"13,133.00",100.00
1,ExpressScripts_NYSE_ESRX,0.31,112.34,"11,234.00",100.00
2,TrinityIndustries_NYSE_TRN,0.06,23.44,"2,344.00",100.00
3,Trex_NYSE_TREX,0.15,53.45,"5,345.00",100.00
4,ACM_Research_Inc,0.12,44.66,"4,466.00",100.00


Then we run a valuation immediately after the stock split. We can see that the units have changed but the PV remains the same.

In [29]:
# Run a "post corporate action" process to automatically adjust prices for instruments which have undergone corporate actions.
# These updated prices are loaded into the quotes store with a 4am timestamp.  This distinguishs them from the official "eod" prices loaded in the previous section
# (For the purposes of this example the post-split price of 26.725 has been calculated externally.)

load_corporate_action_split_price("BBG000BTGM43", 26.725, "2018-05-21T04:00:00Z", scope)

In [30]:
run_portfolio_valuation(portfolio_code, "2018-05-21T08:00:00Z", scope)

,Instrument/default/Name,Proportion(Valuation/PvInReportCcy),Quotes/Price,Sum(Valuation/PvInReportCcy),Sum(Holding/default/Units)
0,Apple_Nasdaq_AAPL,0.36,131.33,"13,133.00",100.00
1,ExpressScripts_NYSE_ESRX,0.31,112.34,"11,234.00",100.00
2,TrinityIndustries_NYSE_TRN,0.06,23.44,"2,344.00",100.00
3,Trex_NYSE_TREX,0.15,26.73,"5,345.00",200.00
4,ACM_Research_Inc,0.12,44.66,"4,466.00",100.00


### 4.5. Bonus Issue

A bonus issue is essentially a stock dividend, the company will reward shareholders by handing out extra shares to current shareholders. In our example the company ACM Research will reward each shareholder with an additional share. This results in multiplying their holdings by 2. We thus have an input factor of 1 and an output factor of 2 while keeping the cost equal. This means that for every 1 share, we receive 2 shares. This will double the position and not affect the cost basis as our output cost factor is 1.

In [31]:
dataset = ca_data.iloc[4]

transitions = [
    lm.CorporateActionTransition(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Figi": dataset["input_instrument_figi"]
            },
            units_factor=float(dataset["input_units_factor"]),
            cost_factor=float(dataset["input_cost_factor"]),
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Figi": dataset["output_instrument_figi"]
                },
                units_factor=float(dataset["output_units_factor"]),
                cost_factor=float(dataset["output_cost_factor"]),
            )
        ],
    )
]

bonus_issue_request = lm.UpsertCorporateActionRequest(
    corporate_action_code=dataset["code"],
    announcement_date=to_datetime(dataset["announcement_date"]),
    ex_date=to_datetime(dataset["ex_date"]),
    record_date=to_datetime(dataset["record_date"]),
    payment_date=to_datetime(dataset["payment_date"]),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[bonus_issue_request]
)

During a bonus issue, a company will provide shareholders with extra shares in the company. On the announcement date we can see that we hold 100 shares in ACM Research.

In [32]:
get_portfolio_holdings("03/03/2022")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,34.00,34.00,"1,895.93",USD,"1,895.93"
3,Trex_NYSE_TREX,P,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,100.00,100.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,P,33.00,33.00,738.07,USD,738.07
6,Cigna_NYSE_CI,P,24.00,24.00,"7,601.00",USD,"7,601.00"
7,USD,B,17.00,17.00,17.00,USD,17.00


On the payment date, we can see that we hold 200 shares, we have received 100 shares. This is since the output factor was 2 in our data. Also note that we did not have to pay for these shares, so our cost amount remains unchanged.

In [33]:
get_portfolio_holdings("23/03/2022")

,properties.Instrument/default/Name.value.label_value,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,Apple_Nasdaq_AAPL,P,100.00,100.00,"4,375.00",USD,"4,375.00"
1,ExpressScripts_NYSE_ESRX,P,100.00,100.00,"7,601.00",USD,"7,601.00"
2,TrinityIndustries_NYSE_TRN,P,34.00,34.00,"1,895.93",USD,"1,895.93"
3,Trex_NYSE_TREX,P,200.00,200.00,"2,783.00",USD,"2,783.00"
4,ACM_Research_Inc,P,200.00,200.00,200.00,USD,200.00
5,Arcosa_NYSE_ACA,P,33.00,33.00,738.07,USD,738.07
6,Cigna_NYSE_CI,P,24.00,24.00,"7,601.00",USD,"7,601.00"
7,USD,B,17.00,17.00,17.00,USD,17.00
